In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
import numpy as np 
import pandas as pd 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import math
%matplotlib inline
from sklearn import model_selection, preprocessing
from sklearn.linear_model import LinearRegression
import xgboost as xgb
color = sns.color_palette()

In [ ]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv("../input/test.csv")
print ('train_df',train_df.shape)
print ('test_df',test_df.shape)

In [ ]:
train_df.head()

In [ ]:
## Describe the output field
print(train_df['price_doc'].describe())
sns.distplot(train_df['price_doc'])


In [ ]:
#The dependent variable is skewed . The easiest would be to take a log tranformation
train_df['LogAmt']=np.log(train_df.price_doc+1.0)
print(train_df['LogAmt'].describe())
sns.distplot(train_df['LogAmt'])

In [ ]:
#do a scatter plot to see if there are any outliers in the data.

plt.figure(figsize=(8,6))
plt.scatter(range(train_df.shape[0]), np.sort(train_df.price_doc.values),color=color[2])
plt.xlabel('index', fontsize=12)
plt.ylabel('price', fontsize=12)
plt.show()


In [ ]:
#get the count of different data types

train_df = pd.read_csv("../input/train.csv", parse_dates=['timestamp'])
dtype_df = train_df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df.groupby("Column Type").aggregate('count').reset_index()


In [ ]:
#explore the number of missing values in each column.

missing = train_df.isnull().sum(0).reset_index()
missing.columns = ['column', 'count']
missing = missing.sort_values(by = 'count', ascending = False).loc[missing['count'] > 0]
missing['percentage'] = missing['count'] / float(train_df.shape[0]) * 100
ind = np.arange(missing.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(10,18))
rects = ax.barh(ind, missing.percentage.values, color='g')
ax.set_yticks(ind)
ax.set_yticklabels(missing.column.values, rotation='horizontal')
ax.set_xlabel("Precentage of missing values %")
ax.set_title("Number of missing values in each column")
plt.show()



In [ ]:
# convert obj to num
# train: 
#yes & no 
YNlist = ['culture_objects_top_25', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion',
          'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion',
          'detention_facility_raion', 'water_1line', 'big_road1_1line', 'railroad_1line']
for c in YNlist :
    train_df.ix[train_df[c]=='yes', c]=1
    train_df.ix[train_df[c]=='no', c]=0
###ecology column
train_df.ix[train_df["ecology"]=='poor', "ecology"]=0
train_df.ix[train_df["ecology"]=='satisfactory', "ecology"]=1
train_df.ix[train_df["ecology"]=='good', "ecology"]=2
train_df.ix[train_df["ecology"]=='excellent', "ecology"]=3
train_df.ix[train_df["ecology"]=='no data', "ecology"]=1
### product_type column
train_df.ix[train_df["product_type"]=='Investment', "product_type"]=1
train_df.ix[train_df["product_type"]=='OwnerOccupier', "product_type"]=0

In [ ]:
# convert obj to num
# test :  
#yes & no 
for i in YNlist :
    test_df.ix[test_df[i]=='yes', i]=1
    test_df.ix[test_df[i]=='no', i]=0
###ecology column
test_df.ix[test_df["ecology"]=='poor', "ecology"]=0
test_df.ix[test_df["ecology"]=='satisfactory', "ecology"]=1
test_df.ix[test_df["ecology"]=='good', "ecology"]=2
test_df.ix[test_df["ecology"]=='excellent', "ecology"]=3
test_df.ix[test_df["ecology"]=='no data', "ecology"]=1
### product_type column
test_df.ix[test_df["product_type"]=='Investment', "product_type"]=1
test_df.ix[test_df["product_type"]=='OwnerOccupier', "product_type"]=0

In [ ]:
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
        if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)
test_df = DataFrameImputer().fit_transform(test_df)


In [ ]:
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]],index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)
train_df = DataFrameImputer().fit_transform(train_df)

In [ ]:
# fit the model
y = train_df.price_doc
train_X = train_df.drop(["price_doc","id","timestamp","sub_area"],axis=1)

linearRegression = LinearRegression()

linearRegression.fit(train_X, y)
print(linearRegression.intercept_)
print(linearRegression.coef_)



In [ ]:
from sklearn import metrics
test_X = test_df.drop(["id","timestamp","sub_area"],axis=1)
y_pred = linearRegression.predict(test_X)
print (y_pred)
print (y_pred.shape)

In [ ]:
submission_df = pd.DataFrame({'id':test_df.id, 'price_doc':y_pred}).set_index('id').to_csv('sub.csv')
import os
cwd = os.getcwd()
print (cwd)